### Brain extraction(Skull stripping)

**Learning outcomes:**
- How to do brain extraction using registration.

In [1]:
%matplotlib inline

import os
from help_functions import *

import ants
import SimpleITK as sitk

print(f'AntsPy version = {ants.__version__}')
print(f'SimpleITK version = {sitk.__version__}')

AntsPy version = 0.4.2
SimpleITK version = 2.3.1


In [2]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
print(f'project folder = {BASE_DIR}')

project folder = c:\Users\Windows\Desktop\GBM_Segmentation


#### Raw Image

In [3]:
raw_img_path = os.path.join(BASE_DIR,'Brain_Extraction', 'sub-01', 'anat', 'sub-01_T1w.nii')
raw_img_ants = ants.image_read(raw_img_path, reorient='IAL')

explore_3D_array(arr=raw_img_ants.numpy(), cmap='nipy_spectral')

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

### Template based method (Native space)

#### Template Image

In [4]:
template_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'mni_icbm152_t1_tal_nlin_sym_09a.nii')
template_img_ants = ants.image_read(template_img_path, reorient='IAL')

explore_3D_array(arr = template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

#### Brain Mask of the template

In [6]:
mask_template_img_path = os.path.join(BASE_DIR, 'Brain_Extraction', 'Template', 'mni_icbm152_nlin_sym_09a', 'mni_icbm152_t1_tal_nlin_sym_09a_mask.nii')
mask_template_img_ants = ants.image_read(mask_template_img_path, reorient='IAL')

explore_3D_array(mask_template_img_ants.numpy())

interactive(children=(IntSlider(value=94, description='SLICE', max=188), Output()), _dom_classes=('widget-inte…

In [7]:
np.unique(mask_template_img_ants.numpy())

array([0., 1.], dtype=float32)

#### Register template to raw image

In [8]:
transformation = ants.registration(
    fixed=raw_img_ants,
    moving=template_img_ants, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [00000223B08B01D0,00000223B08B8D30,1] -m mattes[00000223B08B01D0,00000223B08B8D30,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[00000223B08B01D0,00000223B08B8D30,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [C:\Users\Windows\AppData\Local\Temp\tmp_ad2onjy,00000223B08B8010,00000223B08B6E60] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1


In [9]:
print(transformation)

{'warpedmovout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (320, 300, 208)
	 Spacing    : (0.8, 0.8, 0.8)
	 Origin     : (87.6611, -140.5019, -153.1909)
	 Direction  : [-0.0108 -0.0341 -0.9994  0.0195  0.9992 -0.0343  0.9998 -0.0199 -0.0101]
, 'warpedfixout': ANTsImage (IAL)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (189, 233, 197)
	 Spacing    : (1.0, 1.0, 1.0)
	 Origin     : (98.0, -98.0, -72.0)
	 Direction  : [ 0.  0. -1.  0.  1.  0.  1.  0.  0.]
, 'fwdtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy1Warp.nii.gz', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy0GenericAffine.mat'], 'invtransforms': ['C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy0GenericAffine.mat', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy1InverseWarp.nii.gz']}


In [10]:
registered_img_ants = transformation['warpedmovout']

explore_3D_array_comparison(
    arr_before=raw_img_ants.numpy(), 
    arr_after=registered_img_ants.numpy()
)

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Apply the generated transformations to the mask of template

In [11]:
brain_mask = ants.apply_transforms(
    fixed=transformation['warpedmovout'],
    moving=mask_template_img_ants,
    transformlist=transformation['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

['-d', '3', '-i', '00000223964207D0', '-o', '0000022396420890', '-r', '0000022396420910', '-n', 'nearestNeighbor', '-t', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy1Warp.nii.gz', '-t', 'C:\\Users\\Windows\\AppData\\Local\\Temp\\tmp_ad2onjy0GenericAffine.mat']


In [12]:
explore_3D_array(brain_mask.numpy())

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

In [13]:
explore_3D_array_with_mask_contour(raw_img_ants.numpy(), brain_mask.numpy())

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

In [14]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')

explore_3D_array_with_mask_contour(raw_img_ants.numpy(), brain_mask_dilated.numpy())

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…

#### Generate brain masked

In [15]:
masked = ants.mask_image(raw_img_ants, brain_mask_dilated)

explore_3D_array(masked.numpy())

interactive(children=(IntSlider(value=159, description='SLICE', max=319), Output()), _dom_classes=('widget-int…